## 5. Interpretability Analysis (SHAP & LIME)

In [ ]:
!pip install lime

In [ ]:
# 5_interpretability_analysis.ipynb

import numpy as np
import pandas as pd

import shap
import lime
import lime.lime_tabular

from sklearn.model_selection import train_test_split

In [ ]:
# 5.1 Load the best model (assume XGBoost + SMOTE was best in previous step)
import joblib
best_model = joblib.load("xgb_smote.pkl")

In [ ]:
# 5.2 Load data to compute explanations (we’ll use the original test set)
df = pd.read_parquet("creditcard_engineered.parquet")
X = df.drop(columns="Class")
y = df["Class"]

# Re-apply the same scaler as during training
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # In practice, load the scaler used during training

_, X_te, _, y_te = train_test_split(X_scaled, y, test_size=0.30, random_state=42, stratify=y)

# 5.3 SHAP analysis (global + local)
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_te)


In [ ]:

# 5.3.1 Global feature importance plot (summary)
plt.figure(figsize=(8, 6))
shap.summary_plot(shap_values, X_te, feature_names=X.columns, plot_type="bar", max_display=15)
plt.title("SHAP Feature Importance (Top 15)")
plt.show()

In [ ]:
# 5.3.2 Local explanation for a single instance
idx = 0  # examine the first test sample
shap.initjs()
shap.force_plot(
    explainer.expected_value, shap_values[idx], X_te[idx], feature_names=X.columns, matplotlib=True
)

In [ ]:
# 5.4 LIME analysis (local)
#    Note: We need to train a wrapper around our XGBoost model with scaler, since LIME expects raw feature values
class WrappedModel:
    def __init__(self, model, scaler):
        self.model = model
        self.scaler = scaler

    def predict_proba(self, X_raw):
        X_scaled = self.scaler.transform(X_raw)
        return self.model.predict_proba(X_scaled)

wrapped = WrappedModel(best_model, scaler)

# Use LIME Tabular Explainer
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=scaler.inverse_transform(X_te),
    feature_names=X.columns,
    class_names=["Legit", "Fraud"],
    mode="classification"
)

lime_exp = lime_explainer.explain_instance(
    data_row=scaler.inverse_transform(X_te)[idx],
    predict_fn=wrapped.predict_proba,
    num_features=10
)
lime_exp.show_in_notebook(show_table=True)

# 5.5 Save SHAP values and LIME explanation (optional)
np.save("shap_values.npy", shap_values)
lime_exp.save_to_file("lime_exp.html")
